# El modelo de resultados potenciales

Para entender la econometría necesitas comprender el multiverso.

El multiverso se basa en la interpretación de Hughes Everett (1957) de la mecánica cuántica (Busch, *et al*., 2007). Pero la interpretación popular es que cada decisión que tomamos genera un universo nuevo.

Hay un universo en el que aún estás con tu ex.

Le vamos a llamar a ese universo un **contrafactual**, porque se opone a la realidad. Naturalmente, el hubiera no existe y no podemos obtener datos de los contrafactuales. Sólo podemos imaginarlo.

## En un universo, fuiste a Harvard

¿Habrías tenido mejor vida si hubieras atendido clases en Harvard?

Medir la felicidad en la vida es muy difícil. Nos tendremos que conformar con tus ingresos. ¿Tendrías hoy mejores ingresos si hubieras ido a Harvard?

No podemos comparar los ingresos de un egresado de Harvard con alguien que no fue. Los estudiantes de Harvard tienen (suponemos) más ingresos, mejores conexiones y un estilo de vida diferente a los que no asistieron. Decimos a esto que son **fundamentalmente** diferentes.

## El error de novatos

Si cometes este error estás *banneado* de por vida de mi clase.

Supongamos que deseas saber el **efecto** que tiene entrar en una escuela más selectiva en los ingresos. Decides entonces que es buena idea evaluar la diferencia entre los ingresos de las personas que fueron a una escuela selectiva y los egresados de una no selectiva.

$$
E[Y_1]-E[Y_0]
$$

La $E$ denota la **esperanza** o valor esperado de lo que está entre paréntesis. Normalmente denotamos con $Y$ el **resultado** o la variable en la que esperamos ver un efecto. En este caso es el ingreso. 

El subíndice $1$ o $0$ nos indica el **grupo** al que pertenece la variable. Normalmente $1$ indica el **grupo de tratamiento** y $0$ el **grupo de control**.

Tratamiento y control son parte del lenguaje en los estudios clínicos. Cuando quieres hacer un experimento para saber si una medicina funciona, divides a tus sujetos en dos grupos y a uno le aplicas la medicina y al otro no. Luego mides los resultados.

Imaginemos que esta base de datos identifica a 180 alumnos, la mitad de ellos entró a una escuela selectiva y la otra mitad no lo hizo. ¿Cuál es el valor que representa la ecuación de arriba?

In [29]:
import pandas as pd
import numpy as np
import random

# Creando un diccionario con 'id' como claves y range(179) como valores
alumnos = {'id': list(range(180))}

# Creando el DataFrame
df = pd.DataFrame(alumnos)

# La semilla ayuda a tener el mismo resultado
random.seed(42)
np.random.seed(42)

# La habilidad es aleatoria. 
df['habilidad'] = [np.random.normal(0, 1) for _ in range(len(df))]
ruido_habilidad = np.random.normal(0, 0.2, len(df))

# Añadiendo la columna 'selectivas' con una elección aleatoria de 0 o 1 para cada fila
df['selectivas'] = (df['habilidad'] + ruido_habilidad > 0.5).astype(int)


# Añadiendo la columna 'ingresos' con el cálculo especificado
df['ingresos'] = 1000 + df['selectivas'] * 250 + df['habilidad'] * 250 + np.random.normal(0, 280, len(df))

# Mostrando las primeras filas del DataFrame
df.head()


id  habilidad  selectivas     ingresos
0   0   0.496714           1  1519.595562
1   1  -0.138264           0  1394.600820
2   2   0.647689           0  1131.469293
3   3   1.523030           1  1743.236746
4   4  -0.234153           0  1134.701974

El código de arriba le podrá dar una pista a los más avanzados del problema. Los ingresos en esta tabla dependen de asistir a una escuela selectiva. Veamos la diferencia de promedios.

In [30]:
df[df['selectivas'] == 1]['ingresos'].mean() - df[df['selectivas'] == 0]['ingresos'].mean()

676.1391841200144

Los datos nos dicen que hay una diferencia de 676 (digamos que son miles de) dólares. ¿Es esto cierto?

## Sesgo de Selección

La comparación que hicimos tiene un problema. ¿Lo notaste?

Los estudiantes de escuelas selectivas son **fundamentalmente diferentes** a los que no asisten a este tipo de instituciones. Lo podemos observar en la línea en la que creamos la **dummy** de personas que entraron a escuelas selectivas.

Los egresados de escuelas selectivas tienen padres con mayores ingresos en promedio.

Los ingresos de los padres influyen en la alimentación, el acceso a cuidados, la educación extracurricular y otros aspectos que traen más posibilidades de entrar a la universidad.

Se le conoce como sesgo de selección.

[Sesgo de seleccion](../../figures/sesgo_seleccion.png)

En otras palabras, si vas a una zona rica en Nueva York tendrás más probabilidad de encontrar a un futuro estudiante de Harvard que si vas a una zona pobre en Puerto Rico (el territorio con tasa de pobreza más alta en EE.UU.).

## Rockstars de la economía: Alan Krueger

Alan Krueger fue una leyenda de la economía.

Fue un destacado académico, y secretario asistente del Tesoro para la política económica en la administración de Barack Obama. Fue autor de múltiples artículos influyentes donde innovó en el uso de **experimentos naturales**.

Krueger falleció en su casa en Marzo de 2019.

Su coautor David Card comentó en 2021 que, de haber estado vivo, no había lugar a dudas de que habría compartido con él el premio Nobel de Economía de ese año.

## El beneficio de entrar a una escuela más selectiva

El artículo de Krueger es un gran ejemplo de sesgo de selección y cómo evitarlo.

La comparación no debe ser entre aquellos que entraron a escuelas selectivas y los que no. La comparación debe ser a **las mismas personas en universos paralelos**.

Imaginemos que Alicia logró entrar a Harvard, pero Bernardo no. Alicia sabe tres idiomas, tuvo tutorías personalizadas durante la preparatoria y pasaba las tardes en clases extracurriculares. Bernardo trabaja por las tardes para apoyar a su familia, tiene buenas calificaciones, pero no ha tomado tutorías extra. Alicia tiene computadora en casa y buen acceso a internet, Bernardo tiene una computadora descompuesta y no hay internet en su casa.

**No podemos comparar a Alicia con Bernardo y pensar que las diferencias en sus ingresos vienen de su inscripción a Harvard.**

Lo ideal sería comparar a Alicia en el universo 1, donde si entró a Harvard, con Alicia del universo 0, donde no entró a Harvard. Digamos que $Y_i$ son los ingresos de Alicia. Podemos incluir un subíndice más para indicar el universo.

$Y_{1i}$ indica los ingresos de Alicia en el universo 1 y $Y_{0i}$ en el universo 0.

Finalmente, $D_i$ es una **dummy** que indica si Alicia entró a Harvard ($D_i = 1$) o no ($D_i =0$).

El ingreso de Alicia entonces es

$$
Y_i = \begin{cases} Y_{1i} \qquad \text{si } D_i=1\\ Y_{0i} \qquad \text{si } D_i=0\end{cases}
$$

El ingreso de Alicia en el universo 0 no lo podemos observar, pues está en otro universo donde ella no entró a Harvard. Aún así podemos denotar el efecto de haber entrado a esa universidad en sus ingresos como $Y_{1i} -Y_{0i}$, por lo tanto el ingreso de Alicia en función de $D_i$ sería

$$
Y_i = Y_{0i} + D_i(Y_{1i} - Y_{0i})
$$

Escribir este efecto aún cuando no se puede observar es un ejercicio que nos ayuda a entender el sesgo que se genera al comparar grupos diferentes. 

Cuando $D_i = 1$, los ingresos de Alicia son $Y_{1i}$ y cuando $D_i = 0$ sus ingresos son $Y_{0i}$. Para observar esto no se necesita imaginación. Pero $E[Y_{0i} | D_i = 1]$ es un **contrafactual**, es algo que no pasó, pero que podría pasar. Son los ingresos de todas las personas en situación similar a la de Alicia ($D_i = 1$), **si no hubiéran entrado a Harvard**.

## Descubriendo el sesgo de selección

Hagamos un truco. Nota que

$$
E[Y_1]-E[Y_0] = E[Y_{1i} | D_i = 1] - E[Y_{0i} | D_i = 0]
$$

Son los ingresos observados. Pero lo que realmente necesitamos comparar es 

$$
E[Y_{1i} | D_i = 1] - E[Y_{0i} | D_i = 1]
$$

Se le llama el tratamiento promedio en el tratado. En inglés es *Average treatment on the treated* y lo verás con las siglas ATT.

La ecuación es diferente a la que comenzamos. La diferencia es sutil. Nota que $E[Y_{0i} | D_i = 1]$ es un contrafactual. Son los ingresos que tendrían las personas que entraron a Harvard si no hubieran entrado. Si pudiéramos introducirlas en la ecuación 1, tendríamos dos partes:

$$
E[Y_{i} | D_i = 1] - E[Y_{i} | D_i = 0] = \underbrace{E[Y_{1i} | D_i = 1] - E[Y_{0i} | D_i = 1]}_\text{ATT} + \underbrace{E[Y_{0i} | D_i = 1] -E[Y_{0i} | D_i = 0]}_\text{Sesgo de selección}
$$

Es un truco sucio, ¿lo notaste? Solo incluí el contrafactual como suma y como resta otra vez para no alterar la ecuación. Hacer esto revela que la ecuación inicial incluye el sesgo de selección.

## El experimento aleatorio ideal

Hay una manera de eliminar el sesgo de selección.

Los ensayos controlados aleatorizados (RCTs) son experimentos donde nos aseguramos de eliminar todas las variables que podrían afectar nuestro resultado. Los puedes encontrar en inglés como RCTs: *Randomized controlled trials. S*on el *gold standard* de los estudios científicos.

Los resultados de un RCT son considerados causales.

Un ejemplo de experimento en economía es el que hicieron Miguel & Kremer (2004) para encontrar los efectos de tratar contra las lombrices a alumnos en las escuelas en Kenya. Al eliminar los parásitos no sólo mejoraron la salud de los alumnos, también aumentaron la participación en clase de las escuelas **y de escuelas cercanas**.

Para hacer un experimento necesitas:

- **Controlar las condiciones del estudio.** En la medida de lo posible, elige hacer tu estudio en grupos que sean comparables y homogéneos.
- **Divide en dos grupos: tratamiento y control.** El grupo de tratamiento es donde esperamos ver el efecto, el grupo de control sirve para comparar los resultados.
- **Asigna los grupos aleatoriamente.** Todos los participantes en el estudio deben tener la misma probabilidad de pertenecer a cualquier grupo.
- Sigue y analiza tus resultados. Verifica que todo salió de acuerdo al plan y analiza los datos. Veremos más sobre esto a lo largo del libro.

Cuando un experimento está bien diseñado, no se necesitan modelos estadísticos demasiado complejos.

Desafortunadamente, los experimentos no siempre son posibles de ejecutar. Requieren de planeación, tiempo y recursos que no siempre escalan.

Aún así, diseñar el experimento ideal para identificar el efecto que buscamos es un paso esencial en el diseño de nuestra estrategia de identificación.

## Experimento ideal y experimentos naturales

El experimento ideal requiere que imaginemos cómo seguiríamos los pasos para hacer un experimento si no tuviéramos ninguna limitación de recursos, tiempo o incluso ética.

Por ejemplo, el estudio de Dale & Krueger (2002) requeriría que tomáramos a un grupo de estudiantes y mandáramos de manera aleatoria a la mitad a Harvard y registrar los ingresos de los alumnos en ambos grupos al salir.

Pensar en el experimento ideal soluciona la mitad del problema.

Cuando entiendes cuál es el experimento ideal, es más fácil entender la estrategia que debes usar con las limitantes de la vida real. Dale y Krueger no usaron un experimento para encontrar los efectos de entrar a las escuelas selectivas. Lo que hicieron fue una estrategia ingeniosa para que los datos funcionaran como si hubieran hecho un experimento.

Usaron un experimento natural.

Los experimentos naturales son situaciones que encontramos que generan condiciones para analizar un fenómeno. Por ejemplo, Dale & Krueger (2002) aprovecharon a los alumnos que fueron aceptados en escuelas selectivas, pero decidieron no ingresar por motivos externos. Este es un grupo que si se puede comparar con quienes entraron a las universidades selectivas.

No encontraron diferencias significativas por entrar en una universidad selectiva.

Hay una excepción: los alumnos con menos ventajas en su entorno familiar si se beneficiaron de atender a una escuela selectiva. Esto es porque la universidad es una forma de hacer conexiones personales que pueden traer ventajas para toda la vida. Los alumnos con mayor ventaja en su contexto familiar ya tienen acceso a estas conexiones, pero para los menos aventajados, la universidad es una oportunidad significativa.

## Diseña tu experimento ideal con Inteligencia Artificial

Diseñar el experimento ideal es el primer paso para encontrar la estrategia de identificación.

Para diseñar el experimento natural necesitamos:

- Encontrar la relación causal que nos interesa.
- Reducir las variables a indicadores para medirlas.

El resto lo podemos hacer con ayuda de Inteligencia Artificial.

Usaremos chatGPT para este ejercicio.

Algunas advertencias sobre chatGPT:

- Los modelos grandes de lenguaje (LLM) como chatGPT funcionan como una predicción de la siguiente frase que viene en el texto. Es como el predictor de texto de tu teléfono, pero más avanzado.
- El resultado que obtengas depende del modelo que estés usando (yo muestro el resultado de chatGPT 4, que es el mejor en el mercado en el momento en que escribo esto).
- La mejor manera de obtener buenos resultados con chatGPT es interactuando con lo que te arroja. Pídele modificaciones o agrega información para que corrija, no te quedes con lo primero que te arroja.

Usa el siguiente comando. Puedes copiar y pegar directamente en chatGPT.

```
Eres el experto en econometría e inferencia causal.

Diseña el experimento ideal para identificar efecto sobre las variables que te diré a continuación.

Un experimento ideal es una descripción detallada de un experimento que se podría hacer para obtener efectos causales, sin reparos en los recursos, tiempo o dilemas éticos que pueda causar.

Te voy a describir las variables que deseo estudiar y su relación causal que busco identificar. 

Estás listo?
```

 A continuación tienes que indicarle las variables que deseas medir. Por ejemplo, yo escribí esto para pedir indicaciones sobre el experimento ideal de las universidades selectivas.

 ```
 Deseo conocer el efecto de asistir a una universidad selectiva en los ingresos
 
 ```

 Lee con atención el resultado. Si consideras que lo que describe es algo viable, deberías intentarlo.

Con más experiencia, es buena idea que intentes diseñar estos experimentos ideales por tu cuenta. No sólo es un gran ejercicio mental, probablemente te dará la respuesta sobre la estrategia que debes usar en tu proyecto.

## Apéndice

### Notación

La $E(\cdot)$ se lee como el *valor esperado.* Es un concepto más general que simplemente el promedio, pues aplica a más que sólo números.

El valor esperado (o esperanza)

## Referencias

Berg Dale, S. & Krueger A. B. (2002). Estimating the Payoff to Attending a More Selective College: An Application of Selection on Observables and Unobservables. *The Quarterly Journal of Economics*

Busch, P., Heinonen, T., & Lahti, P. (2007). Heisenberg’s uncertainty principle. Physics Reports, 452(6), 155-176. https://doi.org/10.1016/j.physrep.2007.05.006

Everett, H., III. (1957). "Relative State" Formulation of Quantum Mechanics. *Reviews of Modern Physics, 29*(3), 454. https://doi.org/10.1103/RevModPhys.29.454

Miguel, E. & Kremer, M. (2003). Worms: Identifying Impacts on Education and Health in the Presence of Treatment Externalities. *Econometrica* 72(1). p. 159-217. https://doi.org/10.1111/j.1468-0262.2004.00481.x